In [ ]:
!pip install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.1 MB/s eta 0:00:00


In [ ]:
!pip install json_repair

In [ ]:
!pip install openai -U

In [ ]:

def persian_to_finglish(text):
    mapping = {
        'ا': 'a', 'آ': 'aa', 'ب': 'b', 'پ': 'p', 'ت': 't', 'ث': 's', 'ج': 'j',
        'چ': 'ch', 'ح': 'h', 'خ': 'kh', 'د': 'd', 'ذ': 'z', 'ر': 'r', 'ز': 'z',
        'ژ': 'zh', 'س': 's', 'ش': 'sh', 'ص': 's', 'ض': 'z', 'ط': 't', 'ظ': 'z',
        'ع': 'a', 'غ': 'gh', 'ف': 'f', 'ق': 'gh', 'ک': 'k', 'گ': 'g', 'ل': 'l',
        'م': 'm', 'ن': 'n', 'و': 'v', 'ه': 'h', 'ی': 'i', 'ء': '', ' ': ' ',
        'ً': '', 'ٌ': '', 'ٍ': '', 'َ': 'a', 'ُ': 'o', 'ِ': 'e', 'ّ': '', 'ٔ': ''
    }

    result = ""
    for ch in text:
        result += mapping.get(ch, ch)
    return result

def persian_to_finglish_list(text_list):
    return [persian_to_finglish(text) for text in text_list]

In [ ]:
from openai import OpenAI
import json_repair

def json_decode(input_data):
    return json_repair.repair_json(input_data, ensure_ascii=False, return_objects=True)

client = OpenAI(api_key='your token here')
def persian_sentences_to_finglish(sentences: list):
    try:
        prompt = "sentences = [\n" + "\n".join([f'    "{sentence}"' for sentence in sentences]) + "\n] Output:"

        messages =[{"role":"system","content":"""
        Please convert the following list of Persian sentences into Finglish.
        The output should be exactly in the same format as the input list.
        Make sure to distinguish between short and long vowels.
        Example:

        Input:
        sentences = [
            " حالت خوبه؟ ",
            "سلام چه خبر؟",
            "خوبی؟",
            "چه کار می‌کنی؟",
            "اسلام و ایران"
        ]

        Output:
        { "sentences" : [
            "haalet khoobe?",
            "salaam che khabar?",
            "khoobi?",
            "che kaar mikoni?",
            "eslaam va iran"
        ]}

        Now, convert the following list of sentences into Finglish:
        """},{"role":"user","content":prompt}]

        response = client.chat.completions.create(
            model="gpt-5-mini",
            messages=messages,
            reasoning_effort="minimal",
            max_completion_tokens=10000,
        )

        content = response.choices[0].message.content
        res = json_decode(content)
        return res["sentences"]
    except Exception as e:
        return []



In [ ]:
persian_sentences_to_finglish(["سرمدی گفت که خیلی حالت خوبه","همچنین ابراهیم و اسماعیل ذبح شدند"])

['sarmadi goft ke kheyli haalet khoobe',
 'hamchenin ebrahim va esmaeil zabh shodand']

In [ ]:
import json

ff={}
with open("partial_results.jsonl", "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                try:
                    record = json.loads(line)
                    ff[record['text']]=record
                except json.JSONDecodeError:
                    continue

print(len(ff))
def find_json_by_text(input_text: str, file_path: str = "partial_results.jsonl"):


    try:
       for record in ff.values():
                if record.get("text") == input_text:
                    return record
    except FileNotFoundError:
        print(f"⚠️ File not found: {file_path}")
    except Exception as e:
        print(f"⚠️ Error reading {file_path}: {e}")
    return None


25113


In [ ]:
find_json_by_text('آب این مکان از چشمه ای تأمین می شود که از دل کوه می جوشد و آب سرد و گوارایی را به رهگذران خسته تقدیم می کند سرسبزی و هوای بسیار مطلوب این محیط باعث شده است که چوپانان این نقطه را برای چرای دام های خود بر گزینند و گله گوسفند و بز خود را در این منطقه بچرانند')

{'text': 'آب این مکان از چشمه ای تأمین می شود که از دل کوه می جوشد و آب سرد و گوارایی را به رهگذران خسته تقدیم می کند سرسبزی و هوای بسیار مطلوب این محیط باعث شده است که چوپانان این نقطه را برای چرای دام های خود بر گزینند و گله گوسفند و بز خود را در این منطقه بچرانند',
 'result': "aab-e in makan az cheshmeh-i ta’min mishavad ke az del-e kuah mi jushad va aab-e sard va govaaraa'i ra be rahgozar-aan-e khaste taqdim mikonad sarsabzi va hava-ye besyaar matloob-e in mohit baes shodeh ast ke chupaanaan in noghte ra baraye chera-ye daam-haa-ye khod bargizand vagele-ye goosfand va booz-e khod ra dar in manteqe becheraand"}

In [ ]:
from huggingface_hub import login
import threading,json

lock = threading.Lock()

from datasets import load_dataset, Audio, DatasetDict, Features, Value

login("your huggingface token here")

ds = load_dataset("saeedzou/PersianVox_NM", split="train[:55%]", keep_in_memory=False)
if "audio" in ds.column_names:
    ds = ds.cast_column("audio", Audio(decode=False))
keep = [c for c in ["audio", "text","id"] if c in ds.column_names]
ds = ds.select_columns(keep)

def proc(ex):

    try:
      mannual = persian_to_finglish_list(ex["text"])

      objects= find_json_by_text(ex['text'][-1])
      if objects:
        print("hit")
        t=[find_json_by_text(text)['result'] for text in ex['text']]
      else:
        print("miss")
        t = persian_sentences_to_finglish(ex["text"])
        if not t:
          t = persian_to_finglish_list(ex["text"])
    except:
      t=ex["text"]
      mannual=ex["text"]

    with lock:
            with open("partial_results.jsonl", "a", encoding="utf-8") as f:
                for a, b in zip(ex['text'], t):
                    f.write(json.dumps({"text": a, "result": b}, ensure_ascii=False) + "\n")


    return {"audio":ex["audio"],"text":ex["text"],"mannual":mannual,"text_gpt5_mini": t}

output_features = Features({
    "audio": Audio(decode=False),
    "mannual": Value("string"),
    "text": Value("string"),
    "text_gpt5_mini": Value("string"),
})

ds_small = ds.map(
    proc,
    batched=True,
    batch_size=20,
    num_proc=5,
    features=output_features,
    desc="Processing text"
)
out = DatasetDict({"train": ds_small})
out.push_to_hub("David-ger/Persian-tts-finglish")
print("✅ Pushed!")